In [38]:
# Try a linear regression model
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

# train and test the model
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)
lr.fit(X_train, y_train)
lr.score(X_test, y_test)

0.6166152922497015

In [52]:
# Import the neural network model
from sklearn.neural_network import MLPRegressor
mlp = MLPRegressor()

# train and test the model
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)
mlp.fit(X_train, y_train)
mlp.score(X_test, y_test)
print(mlp.score(X_test, y_test))

# Import the random forest model
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()

# train and test the model
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)
rf.fit(X_train, y_train)
rf.score(X_test, y_test)
print(rf.score(X_test, y_test))

# Import the gradient boosting model
from sklearn.ensemble import GradientBoostingRegressor
gb = GradientBoostingRegressor()

# train and test the model
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)
gb.fit(X_train, y_train)
gb.score(X_test, y_test)
print(gb.score(X_test, y_test))

# Import the decision tree model
from sklearn.tree import DecisionTreeRegressor
dt = DecisionTreeRegressor()

# train and test the model
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)
dt.fit(X_train, y_train)
dt.score(X_test, y_test)
print(dt.score(X_test, y_test))

# Import the KNN model
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor()

# train and test the model
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)
knn.fit(X_train, y_train)
knn.score(X_test, y_test)
print(knn.score(X_test, y_test))

# Import the SVM model
from sklearn.svm import SVR
svr = SVR()

# train and test the model
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)
svr.fit(X_train, y_train)
svr.score(X_test, y_test)
print(svr.score(X_test, y_test))

# Import the XGBoost model
from xgboost import XGBRegressor
xgb = XGBRegressor()

# train and test the model
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)
xgb.fit(X_train, y_train)
xgb.score(X_test, y_test)
print(xgb.score(X_test, y_test))



-0.3225987489290634
0.5240861558855514
0.23214047765247603
0.6453002910148729
-0.9367850107783138
0.5768242110534658
0.8118657056470845


In [50]:
!pip install lightgbm

   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ----- ---------------------------------- 0.2/1.3 MB 5.3 MB/s eta 0:00:01
   --------------------- ------------------ 0.7/1.3 MB 8.9 MB/s eta 0:00:01
   ------------------------------------- -- 1.2/1.3 MB 9.9 MB/s eta 0:00:01
   ---------------------------------------- 1.3/1.3 MB 9.4 MB/s eta 0:00:00


In [53]:
import hopsworks

# Log in to Hopsworks
project = hopsworks.login()
fs = project.get_feature_store()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/194708
Connected. Call `.close()` to terminate connection gracefully.


In [40]:
# Make the columns lowercase 
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(' ', '_')

# Create the feature group
# fg = fs.create_feature_group("incidents", version=1, description="Incidents in the city of Stockholm", 
#                              primary_key=df.columns.tolist(),)
fg = fs.get_feature_group("incidents", version=1)
fg.insert(df)

Uploading Dataframe: 100.00% |██████████| Rows 1000/1000 | Elapsed Time: 00:05 | Remaining Time: 00:00


Launching job: incidents_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/194708/jobs/named/incidents_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x22dfde27ad0>, None)

In [41]:
query = fg.select_all()
feature_view = fs.get_or_create_feature_view("incidents_view", version=1, description="Incidents in the city of Stockholm",  labels=["duration"], query=query)
X_train, X_val, X_test, y_train, y_val, y_test = feature_view.train_validation_test_split(0.1, 0.1)


Finished: Reading data from Hopsworks, using ArrowFlight (1.39s) 


In [42]:
# Train a linear regression model
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

# train and test the model
lr.fit(X_train, y_train)
metric = lr.score(X_test, y_test)

In [43]:
# We will now upload the linear regression model to the registry
mr = project.get_model_registry()

# The directory will be wine_model
model_dir="stockholm_incidents_model"
if os.path.isdir(model_dir) == False:
    os.mkdir(model_dir)

# Save the model
joblib.dump(lr, model_dir + "/stckhlm_inc_model.pkl")

# Specify the input and output
input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema, output_schema)

# Create an entry
wine_model = mr.python.create_model(
    name="stockholm_incidents_model", 
    metrics={"MSE" : metric},
    model_schema=model_schema,
    description="Stockholm Incident duration Predictor"
)

# Upload the model to the model registry, including all files in 'model_dir'
wine_model.save(model_dir)

Connected. Call `.close()` to terminate connection gracefully.


Uploading: 100.000%|██████████| 1071/1071 elapsed<00:01 remaining<00:00:01,  2.69it/s]
Uploading: 100.000%|██████████| 714/714 elapsed<00:01 remaining<00:00<00:04,  1.07s/it]
Model export complete: 100%|██████████| 6/6 [00:09<00:00,  1.58s/it]                   

Model created, explore it at https://c.app.hopsworks.ai:443/p/194708/models/stockholm_incidents_model/2


Model(name: 'stockholm_incidents_model', version: 2)